In [83]:
#!/usr/bin/env python3
from rosbags.rosbag1 import Reader
from rosbags.serde import deserialize_cdr, ros1_to_cdr
from rosbags.typesys import get_types_from_msg, register_types
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [84]:
add_types = {}
msg_text = Path('/home/eurus/Documents/College/Sem_4/ENAE788M/proj/ws/src/Project_Catch/vicon_bridge/msg/Marker.msg').read_text()
add_types.update(get_types_from_msg(msg_text, 'vicon_bridge/Marker'))
register_types(add_types)
msg_text = Path('/home/eurus/Documents/College/Sem_4/ENAE788M/proj/ws/src/Project_Catch/vicon_bridge/msg/Markers.msg').read_text()
add_types.update(get_types_from_msg(msg_text, 'vicon_bridge/Markers'))
register_types(add_types)

In [85]:
t_count = 0
marker_pos = {t_count:[]}
prev_t = False
# with Reader('/home/eurus/Documents/College/Sem_4/ENAE788M/proj/ws/src/Project_Catch/misc/marker_throw/throw_1.bag') as reader:
with Reader('/home/eurus/Documents/College/Sem_4/ENAE788M/proj/ws/src/Project_Catch/misc/marker_throw/throw_2.bag') as reader:
    for connection, timestamp, rawdata in reader.messages():
        ## The local position message from the PX4
        if connection.topic == '/vicon/markers':
            msg = deserialize_cdr(ros1_to_cdr(rawdata, connection.msgtype), connection.msgtype)
            if(len(msg.markers)):
                prev_t = True
                if(t_count not in marker_pos):
                    marker_pos[t_count] = []
                marker_pos[t_count].append([msg.markers[0].translation.x, msg.markers[0].translation.y, msg.markers[0].translation.z,msg.header.stamp.sec+msg.header.stamp.nanosec*1e-9])
            else:
                if(prev_t):
                    prev_t = False
                    if(len(marker_pos[t_count])<50):
                        marker_pos[t_count] = []
                    else:
                        marker_pos[t_count] = np.array(marker_pos[t_count])
                        t_count += 1
                    

In [86]:
# for i in range(0, len(marker_pos)):
#     marker_ = np.array(marker_pos[i])
#     marker_[:,-1] = marker_[:,-1] - marker_[0,-1]
#     plt.scatter(marker_[:,-1], marker_[:,2])
#     plt.title('Marker Z Position vs Time Index: '+str(i))
#     plt.pause(0.1)

# Kalman Filter 3D parabola fit

In [87]:
throw_ = 1
marker_pos[throw_][1:,-1] = marker_pos[throw_][1:,-1] - marker_pos[throw_][:-1,-1]
marker_pos[throw_][0,-1] = 0

In [88]:
class KalmanFilter3D:
    def __init__(self):
        # Kalman Filter
        # State Vector
        ## x y z az vx vy vz ax ay 
        self.state = np.zeros((9,1))

        # Measurement Matrix
        ## Defines which states are measured
        ## Only position is measured, x y z az
        self.H = np.array([[1,0,0,0,0,0,0,0,0],
                    [0,1,0,0,0,0,0,0,0],
                    [0,0,1,0,0,0,0,0,0],
                    [0,0,0,1,0,0,0,0,0],
                    ])


        # Covariance Matrix
        ## Defines how much we trust the model 
        self.R = np.array([[0.01,0,0,0],
                    [0,0.01,0,0],
                    [0,0,0.01,0],
                    [0,0,0,0.01],
                    ])


        # Process Noise Covariance Matrix
        ## Defines the noise relationship between states
        self.Q = np.array([[0.01,0,0,0,0,0,0,0,0],
                    [0,0.01,0,0,0,0,0,0,0],
                    [0,0,0.01,0,0,0,0,0,0],
                    [0,0,0,0.01,0,0,0,0,0],
                    [0,0,0,0,0.01,0,0,0,0],
                    [0,0,0,0,0,0.01,0,0,0],
                    [0,0,0,0,0,0,0.01,0,0],
                    [0,0,0,0,0,0,0,0.01,0],
                    [0,0,0,0,0,0,0,0,0.01],
                    ])

        self.state_est = np.zeros((9,1))
        self.P = np.zeros((9,9))
        
        # Gravity along the Z axis
        self.gravity = -9.81
        
    def update(self, dt, measurement):
        
        # State Prediction
        self.state_predict(dt)
        
        # Measurement Update
        
        # Update the Kalman Gain
        ## Update the Kalman Gain based on the covariance and measurement matrix
        self.K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(self.H @ self.P @ self.H.T + self.R)))
        
        measurement = np.vstack((measurement.reshape(3,1), np.ones((1,1))*self.gravity))
        # Update the state estimation using the Kalman Gain
        self.state_est = self.state + np.dot(self.K, measurement - self.H @ self.state)
        
        pass
    
    def state_predict(self, dt):
        #######
        # State Transition Matrix
        ## Defines states are dependent on previous states
        ## The lines are equation of motion s = s0 + v0*t + 0.5*a*t^2
        A = np.array([[1,0,0,0,dt,0,0,0.5*dt**2,0],
                      [0,1,0,0,0,dt,0,0,0.5*dt**2],
                      [0,0,1,0.5*dt**2,0,0,dt,0,0],
                      [0,0,0,1,0,0,0,0,0],
                      [0,0,0,0,1,0,0,dt,0],
                      [0,0,0,0,0,1,0,0,dt],
                      [0,0,0,dt,0,0,1,0,0],
                      [0,0,0,0,0,0,0,1,0],
                      [0,0,0,0,0,0,0,0,1],
                    ])
        ########
        
        # State Update
        ## Predict next state based on state transition matrix
        self.state = np.dot(A,self.state_est)
        
        # Covariance Update
        ## Update the covariance of the model based on the previous state and process noise
        self.P = np.dot(np.dot(A,self.P),A.T) + self.Q

        

In [90]:
ekf = KalmanFilter3D()
lim = 50
for wp, marker_ in enumerate(marker_pos[throw_][:,:]):
    # print(marker_)
    if(wp < lim):
        ekf.update(marker_[-1], marker_[:3])
    else:
        
    
  
  

1251.9092366736372

1690.8550264406897

1684.7393726387331

1680.6627291827979

1678.0452852522872

1678.7871829782262

1679.6799794013543

1676.1198867829444

1675.1238725122123

1677.3918176472637

1677.723744769896

1675.1489939603607

1680.82072269284

1685.424440573301

1679.1998672014558

1681.42225329588

1683.73505876118

1687.7133133436676

1690.7094278081083

1691.3767487712314

1693.237806213734

1696.391421130981

1697.8291562091295

1699.5768851087794

1704.4652603542781

1708.0049097212157



In [79]:
marker_pos[throw_][:,-1]

array([ 0.00000000e+00,  5.09524345e-03, -2.59637833e-04,  5.97000122e-04,
        1.78194046e-03, -4.45842743e-03,  2.38418579e-03,  1.23691559e-03,
       -2.94661522e-03,  2.19297409e-03, -4.43696976e-04, -5.24520874e-05,
       -4.23431396e-04,  3.03506851e-04,  3.10897827e-04,  1.19996071e-03,
       -1.28936768e-03, -1.08432770e-03, -3.05891037e-04,  3.32522392e-03,
       -3.91745567e-03,  1.71732903e-03, -1.48773193e-04, -3.07559967e-04,
        4.53233719e-04,  7.24792480e-05,  1.80244446e-04, -1.57356262e-04,
       -1.08957291e-04,  5.17344475e-03, -9.84644890e-03,  3.90934944e-03,
        1.29532814e-03, -3.10182571e-04,  2.65359879e-04, -1.28531456e-03,
        1.11269951e-03, -4.76837158e-06, -4.52756882e-04,  3.10182571e-04,
       -1.50203705e-04,  1.51324272e-03, -2.95853615e-03,  1.26624107e-03,
        1.42908096e-03, -2.43854523e-03,  1.41072273e-03, -3.30209732e-04,
        1.73568726e-04,  3.76701355e-05,  8.28027725e-04, -1.19924545e-03,
       -7.96318054e-04,  